###  importing relevant libraries

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")

### Discovering Labels

In [2]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [3]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [5]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [6]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [7]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):
    from sklearn.metrics import balanced_accuracy_score
    from sklearn.preprocessing import Normalizer
    
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            m_train=df["MAC"]
            del df["MAC"]
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df = shuffle(df, random_state=42)
            m_test=df["MAC"]
            del df["MAC"]
            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes




            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        #print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            cm.to_csv(graph_name.replace(".pdf",".csv"))
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [8]:
feature= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"MAC",
'Label']


In [9]:
ml_list={"DT" :DecisionTreeClassifier()}



In [10]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv'],
 ['./csvs\\A211103.csv', './csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211130.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]

In [11]:
datasets=[ ['./csvs\\I211116.csv', './csvs\\I211117.csv'],
 ['./csvs\\A211103.csv', './csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211130.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv']]

In [12]:
say=0
dataset="./results13/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=13
            sayac=1
            dataset="./results13/vs/"
            if ii[7:-4] != iii[7:-4]:
                output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
                dataset=f"{ii[7:-4]}-{iii[7:-4]}"
                target_names=target_name(test)
                ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211116-I211117_13    0     1  DT    0.835    0.837   0.833  0.837  0.793  0.827      0    0.6       0.096      8.645
 1  I211116-I211117_13    1     2  DT    0.835    0.836   0.851  0.836  0.805  0.827      0    1.703     0.189      8.085
 2  I211116-I211117_13    2     3  DT    0.836    0.838   0.833  0.838  0.795  0.828      0    1.635     0.189      7.929
 3  I211116-I211117_13    3     4  DT    0.835    0.837   0.833  0.837  0.793  0.827      0    1.653     0.19       7.986
 4  I211116-I211117_13    4     5  DT    0.835    0.837   0.833  0.837  0.794  0.828      0    1.636     0.189      8.009



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211117-I211116_13    0     1  DT    0.887    0.901   0.875  0.901  0.87   0.881      0    1.59      0.186      7.987
 1  I211117-I211116_13    1     2  DT    0.887    0.901   0.876  0.901  0.87   0.882      0    1.577     0.189      7.852
 2  I211117-I211116_13    2     3  DT    0.887    0.9     0.876  0.9    0.87   0.881      0    1.558     0.179      7.863
 3  I211117-I211116_13    3     4  DT    0.888    0.901   0.877  0.901  0.871  0.882      0    1.589     0.17       8.199
 4  I211117-I211116_13    4     5  DT    0.888    0.901   0.877  0.901  0.872  0.882      0    1.553     0.189      8.038



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [13]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv']]

In [15]:
say=0
dataset="./results13/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=13
            sayac=1
            dataset="./results13/vs/"
            if ii[7:-4] != iii[7:-4]:
                output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
                dataset=f"{ii[7:-4]}-{iii[7:-4]}"
                target_names=target_name(test)
                ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211105_13    0     1  DT    0.981    0.927   0.958  0.927  0.934   0.98      0    1.328     0.141      6.174
 1  A211102-A211105_13    1     2  DT    0.981    0.927   0.958  0.927  0.934   0.98      0    1.344     0.141      6.016
 2  A211102-A211105_13    2     3  DT    0.981    0.926   0.958  0.926  0.933   0.98      0    1.112     0.156      6.063
 3  A211102-A211105_13    3     4  DT    0.981    0.928   0.958  0.928  0.935   0.98      0    1.391     0.141      6.031
 4  A211102-A211105_13    4     5  DT    0.981    0.926   0.958  0.926  0.933   0.98      0    1.406     0.141      5.721



  7%|█████▉                                                                             | 2/28 [01:02<13:37, 31.44s/it]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211108_13    0     1  DT    0.986    0.951   0.961  0.951  0.955  0.985      0    1.002     0.203      7.846
 1  A211102-A211108_13    1     2  DT    0.986    0.952   0.961  0.952  0.955  0.985      0    1.344     0.188      7.632
 2  A211102-A211108_13    2     3  DT    0.986    0.953   0.961  0.953  0.956  0.985      0    1.344     0.204      7.649
 3  A211102-A211108_13    3     4  DT    0.987    0.953   0.96   0.953  0.956  0.985      0    1.359     0.188      7.748
 4  A211102-A211108_13    4     5  DT    0.986    0.952   0.96   0.952  0.955  0.985      0    1.234     0.203      8.156



 11%|████████▉                                                                          | 3/28 [02:21<21:12, 50.91s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [ ]:
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  

In [13]:
say=0
dataset="./results13/vs/"
folder(dataset)
for no, ii in enumerate(  ['./csvs\\I211102.csv']):
        for iii in tqdm(['./csvs\\I211118.csv',  './csvs\\I211119.csv',  './csvs\\I211122.csv']):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=13
            sayac=1
            dataset="./results13/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211102-I211118_13    0     1  DT    0.796    0.8      0.75  0.8    0.753  0.785      0    1.846     0.205      7.778
 1  I211102-I211118_13    1     2  DT    0.796    0.799    0.75  0.799  0.752  0.785      0    1.867     0.232      7.684
 2  I211102-I211118_13    2     3  DT    0.795    0.799    0.75  0.799  0.752  0.784      0    1.793     0.188      7.86
 3  I211102-I211118_13    3     4  DT    0.795    0.799    0.75  0.799  0.751  0.784      0    1.902     0.205      8.395
 4  I211102-I211118_13    4     5  DT    0.796    0.799    0.75  0.799  0.752  0.785      0    1.8       0.183      8.482



 33%|████████████████████████████                                                        | 1/3 [01:25<02:51, 85.59s/it]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211102-I211119_13    0     1  DT    0.802    0.806   0.796  0.806  0.767  0.791      0    1.838     0.207      8.577
 1  I211102-I211119_13    1     2  DT    0.802    0.806   0.815  0.806  0.767  0.791      0    1.845     0.268     10.07
 2  I211102-I211119_13    2     3  DT    0.802    0.806   0.829  0.806  0.767  0.791      0    1.877     0.188      8.161
 3  I211102-I211119_13    3     4  DT    0.802    0.806   0.779  0.806  0.767  0.791      0    1.704     0.178      7.764
 4  I211102-I211119_13    4     5  DT    0.802    0.806   0.818  0.806  0.768  0.791      0    1.614     0.154      7.324



 67%|████████████████████████████████████████████████████████                            | 2/3 [02:51<01:26, 86.02s/it]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211102-I211122_13    0     1  DT    0.792    0.772   0.76   0.772  0.732   0.78      0    1.676     0.173      7.413
 1  I211102-I211122_13    1     2  DT    0.791    0.772   0.76   0.772  0.732   0.78      0    1.872     0.174      8.249
 2  I211102-I211122_13    2     3  DT    0.791    0.772   0.76   0.772  0.732   0.78      0    1.878     0.188      8.145
 3  I211102-I211122_13    3     4  DT    0.791    0.772   0.727  0.772  0.732   0.78      0    1.793     0.182      7.966
 4  I211102-I211122_13    4     5  DT    0.792    0.772   0.727  0.772  0.732   0.78      0    1.998     0.262      8.192



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:15<00:00, 85.07s/it]


<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [14]:
say=0
dataset="./results13/vs/"
folder(dataset)
for no, ii in enumerate(  ['./csvs\\I211103.csv',   './csvs\\I211104.csv',   './csvs\\I211108.csv',   './csvs\\I211110.csv',   './csvs\\I211112.csv',   './csvs\\I211115.csv',   './csvs\\I211118.csv',   './csvs\\I211119.csv',   './csvs\\I211122.csv'] ):
        for iii in tqdm(['./csvs\\A211102.csv',   './csvs\\A211105.csv',   './csvs\\A211108.csv',   './csvs\\A211109.csv',   './csvs\\A211110.csv',   './csvs\\A211112.csv',   './csvs\\A211115.csv',   './csvs\\A211116.csv',   './csvs\\A211117.csv',   './csvs\\A211118.csv',   './csvs\\A211122.csv',   './csvs\\A211123.csv',   './csvs\\A211125.csv',   './csvs\\A211206.csv',   './csvs\\A211208.csv',   './csvs\\A211223.csv',   './csvs\\A211225.csv',   './csvs\\A211228.csv',   './csvs\\I211102.csv',   './csvs\\I211103.csv',   './csvs\\I211104.csv',   './csvs\\I211108.csv',   './csvs\\I211110.csv',   './csvs\\I211112.csv',   './csvs\\I211115.csv',   './csvs\\I211118.csv',   './csvs\\I211119.csv',   './csvs\\I211122.csv'] ):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=13
            sayac=1
            dataset="./results13/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211103-A211102_13    0     1  DT    0.801    0.857   0.855  0.857  0.831  0.792      0    1.475     0.147      5.887
 1  I211103-A211102_13    1     2  DT    0.8      0.856   0.855  0.856  0.83   0.792      0    1.38      0.147      6.466
 2  I211103-A211102_13    2     3  DT    0.801    0.856   0.855  0.856  0.83   0.792      0    1.459     0.146      6.681
 3  I211103-A211102_13    3     4  DT    0.8      0.858   0.855  0.858  0.831  0.792      0    1.561     0.153      6.305
 4  I211103-A211102_13    4     5  DT    0.8      0.857   0.841  0.857  0.814  0.792      0    1.471     0.232      6.332



  4%|██▉                                                                                | 1/28 [01:07<30:09, 67.03s/it]

    Dataset               T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ------------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211103-A211105_13    0     1  DT    0.855    0.895   0.875  0.895  0.861  0.848      0    1.393     0.153      6.246
 1  I211103-A211105_13    1     2  DT    0.856    0.896   0.865  0.896  0.847  0.849      0    1.481     0.172      6.494
 2  I211103-A211105_13    2     3  DT    0.855    0.895   0.875  0.895  0.861  0.848      0    1.476     0.169      6.448
 3  I211103-A211105_13    3     4  DT    0.856    0.896   0.876  0.896  0.862  0.849      0    1.435     0.149      6.103
 4  I211103-A211105_13    4     5  DT    0.855    0.896   0.864  0.896  0.846  0.849      0    1.385     0.187      6.427



  7%|█████▉                                                                             | 2/28 [02:14<29:10, 67.32s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [15]:
!shutdown /s /t 360

In [3]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv'],
 ['./csvs\\A211103.csv', './csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211130.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]

In [9]:
say=0
dataset="./results13/vs/"

for no, i in enumerate(datasets):
    for ii in i:
        myfile=f"{dataset}/@{ii[7:-4]}@{ii[7:-4]}CB.csv"
        if os.path.isfile(myfile):
            os.remove(myfile)
            print("Favtutor file deleted using remove() function")
            print("Current files in directory -->",os.listdir())
            print("-----------------------------------------------------")
        else:
            # If it fails, inform the user.
            print("Error: %s file not found" % myfile)

Error: ./results13/vs//@A211102@A211102CB.csv file not found
Error: ./results13/vs//@A211105@A211105CB.csv file not found
Error: ./results13/vs//@A211108@A211108CB.csv file not found
Error: ./results13/vs//@A211109@A211109CB.csv file not found
Error: ./results13/vs//@A211110@A211110CB.csv file not found
Error: ./results13/vs//@A211112@A211112CB.csv file not found
Error: ./results13/vs//@A211115@A211115CB.csv file not found
Error: ./results13/vs//@A211116@A211116CB.csv file not found
Error: ./results13/vs//@A211117@A211117CB.csv file not found
Error: ./results13/vs//@A211118@A211118CB.csv file not found
Error: ./results13/vs//@A211122@A211122CB.csv file not found
Error: ./results13/vs//@A211123@A211123CB.csv file not found
Error: ./results13/vs//@A211125@A211125CB.csv file not found
Error: ./results13/vs//@A211206@A211206CB.csv file not found
Error: ./results13/vs//@A211208@A211208CB.csv file not found
Error: ./results13/vs//@A211223@A211223CB.csv file not found
Error: ./results13/vs//@